<★★★자주 쓰는 것들★★★>
- torch.nn (https://pytorch.org/docs/stable/nn.html)
  - torch.nn.functional (https://pytorch.org/docs/stable/nn.functional.html)
  - torch.nn.Module(https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module)
- torch.util
  - torch.utils.data (https://pytorch.org/docs/stable/data.html)
- torch.tensor(https://pytorch.org/docs/stable/tensors.html)

---

#### 1. 텐서 차원 확장, 감소
확장 방법
- tensor.view(1, -1)
- tensor.unsqueeze()
- tensor[..., np.new_axis]
- tensor[..., 1]

축소 방법
- tensor.reshape(-1)
- tensor.squeeze()
- tensor.flatten()

``` python
# torch.Size([1000, 512, 1, 1]) -> torch.Size([1000, 512])
feas_sim_i = torch.flatten(feas_sim_i, start_dim=1) 
```

----

#### `CLASS torch.nn.Identity(*args, **kwargs)`
- 인수를 구분하지 않는 Placeholder ID 연산자
- 입력과 동일한 텐서를 출력으로 내보내주는 layer다
- 관련 이슈: https://github.com/pytorch/pytorch/issues/9160

In [4]:
import torch
import torch.nn as nn
m = nn.Identity(54, unused_argument1=0.1, unused_argument2=False)
input_ = torch.randn(128, 20)
output = m(input_)
print(output)
print(output.shape)
print(output.size())

tensor([[ 1.5573,  0.0421, -1.8752,  ..., -0.0031,  0.4080,  1.5668],
        [-0.3170,  0.7242, -0.0494,  ..., -0.2612,  0.4649,  0.0139],
        [ 0.7769,  0.2675,  0.8932,  ..., -1.7541, -0.1944, -1.2333],
        ...,
        [ 0.7780, -0.9834, -0.5951,  ...,  1.0033,  2.6036,  1.6209],
        [ 0.3349,  0.8116,  1.4694,  ..., -1.0343,  0.4855, -2.1370],
        [ 0.0744, -0.8772, -0.7785,  ..., -0.5631, -0.5481, -0.3703]])
torch.Size([128, 20])
torch.Size([128, 20])


In [ ]:
class Identity(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__()
    def forward(self, x):
        return x

#### `CLASS torch.nn.AdaptiveAvgPool2d(output_size)`
- 파라미터: Union[int, None, Tuple[Optional[int], Optional[int]]]
- 입력 사이즈 (B, C, H_in, W_in) or (C, H_in, W_in)
- 출력 사이즈 (B, C, S_0, S_1) or (C, S_0, S_1)

In [ ]:
pool = nn.AdaptiveAvgPool2d(1)
feas_sim_i = pool(feas_sim_i) # torch.Size([1000, 512, 7, 7]) -> torch.Size([1000, 512, 1, 1])

In [9]:
import torch
import torch.nn as nn
input = torch.randn(1, 64, 8, 9)

m = nn.AdaptiveAvgPool2d((5, 7))
output = m(input)
print(output.shape)

m = nn.AdaptiveAvgPool2d(7)
output = m(input)
print(output.shape)

m = nn.AdaptiveAvgPool2d((None, 7)) # 앞에 것은 신경 안쓰고 뒤의 크기를 7로
output = m(input) # 클래스를 __call__ 메소드를 정의하여 함수 처럼 쓴다
print(output.shape)

torch.Size([1, 64, 5, 7])
torch.Size([1, 64, 7, 7])
torch.Size([1, 64, 8, 7])


#### `torch.flatten(input, start_dim=0, end_dim=-1)`
- https://pytorch.org/docs/stable/generated/torch.flatten.html

#### `torch.nn.parameter.Parameter(data=None, requires_grad=True)`
- 모듈 파라미터로 간주되는 일종의 Tensor(?)
- 파라미터는 Tensor의 하위 클래스로, Module과 함께 사용될 때 매우 특별한 속성을 갖는다.
- 파라미터 Tensor는 모듈의 속성으로 할당되면, 자동으로 파라미터 리스트에 추가되고, .parameters() 이터레이터로 참조할 수 있다.
- `모델에는 모듈이 있고, 모듈에는 파라미터가 있다`
  
- Tensor를 할당하는 것은 그런 효과는 없다(??)
- 이는 RNN 모델의 숨겨진 상태와 같이 일부 임시(temporary) 상태를 캐시하려고 할 수 있기 때문.
- Parameter() 와 같은 클래스가 없으면 이런 임시 항목도 등록이 된다. 
  `(그렇기 때문에 모듈에서 파라미터라는 것을 따로 뺐다??)`
  
--> 모듈도 클래스고, 파라미터도 클래스다. 모듈과 파라미터 클래스는 서로 소통한다.

- data: 파라미터 텐서  
- requires_grad: 파라미터가 gradient를 필요로 한다면, torch.no_grad() 컨텍스트는 파라미터 생성의 기본 동작에 영향을 미치지 않는다. 
  - 파라미터는 no_grad 모드에서도 여전히 requires_grad=True를 갖는다(???)
  - (생성에는 영향을 미치지 않고, 생성 이후에 해당 문맥 안에서는 requires_grad=False 란 의미인가??)

In [ ]:
'''
Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
==================================================
<class 'torch.nn.parameter.Parameter'> torch.Size([64, 3, 7, 7])
==================================================
BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
==================================================
<class 'torch.nn.parameter.Parameter'> torch.Size([64])
<class 'torch.nn.parameter.Parameter'> torch.Size([64])
==================================================
ReLU(inplace=True)
==================================================
==================================================
MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
==================================================
==================================================
Sequential(
  (0): BasicBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (1): BasicBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)
==================================================
<class 'torch.nn.parameter.Parameter'> torch.Size([64, 64, 3, 3])
<class 'torch.nn.parameter.Parameter'> torch.Size([64])
<class 'torch.nn.parameter.Parameter'> torch.Size([64])
<class 'torch.nn.parameter.Parameter'> torch.Size([64, 64, 3, 3])
<class 'torch.nn.parameter.Parameter'> torch.Size([64])
<class 'torch.nn.parameter.Parameter'> torch.Size([64])
<class 'torch.nn.parameter.Parameter'> torch.Size([64, 64, 3, 3])
<class 'torch.nn.parameter.Parameter'> torch.Size([64])
<class 'torch.nn.parameter.Parameter'> torch.Size([64])
<class 'torch.nn.parameter.Parameter'> torch.Size([64, 64, 3, 3])
<class 'torch.nn.parameter.Parameter'> torch.Size([64])
<class 'torch.nn.parameter.Parameter'> torch.Size([64])
'''